In [128]:
import sys
sys.path.insert(0, '../gofher')

import os
import matplotlib.image as mpimg

from gofher import run_gofher
from visualize import visualize

In [129]:
#name = "IC1199"
name = "NGC4527"
paper_label = "ne"
folder = "figure8"
#bin_size = 4

In [130]:
path_to_input = "..\\..\\spin-parity-catalog-data\\panstarrs\\{}".format(folder)
path_to_output = "..\\..\\..\\"

In [131]:
def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_input,name,"{}_{}.fits".format(name,band))

def get_color_image_path(name):
    return os.path.join(path_to_input,name,"{}_color.jfif".format(name))

In [132]:
if not os.path.exists(path_to_input):
    raise ValueError("The path to the input is not found {} - make sure you update path_to_input".format(path_to_input))

if not os.path.exists(path_to_output):
    raise ValueError("The path output is not found {} - make sure you update ppath_to_output".format(path_to_output))

gal = run_gofher(name,get_fits_path,['g','r','i','z','y'],['i','z','y','r'], paper_label)

#color_image = color = mpimg.imread(get_color_image_path(name))
#visualize(gal,color_image,['r','i','z','y'],paper_label)

#(header,row) = gal.get_verbose_csv_header_and_row(['r','i','z','y'],paper_label)
#print(header)
#print(row)

In [133]:
def make_diff_norm_image_example_for_paper(the_gal):
    #https://walmsley.dev/posts/typesetting-mnras-figures
    #https://support.microsoft.com/en-us/office/add-a-font-b7c5f17c-4426-4b53-967f-455339c564c1
    #error - font not appearing: https://stackoverflow.com/a/26106170
    ## delet file at: /mnt/c/Users/school/.matplotlib
    
    """
    import matplotlib.font_manager
    matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

    fpaths = matplotlib.font_manager.findSystemFonts()
    for i in fpaths:
        f = matplotlib.font_manager.get_font(i)
        print(f.family_name)
    """
    #Nimbus Roman No9 L
    import matplotlib.pyplot as plt
    from matrix import normalize_matrix
    from visualize import create_color_map_class
    import numpy as np
    #import matplotlib
    #matplotlib.rcParams['nbagg.transparent'] = False
    
    # in points - start with the body text size and play around
    
    SMALL_SIZE = 9
    MEDIUM_SIZE = 9
    BIGGER_SIZE = 9

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)
    
    plt.rc('font', family='Nimbus Roman No9 L')

    fig, ax = plt.subplots(nrows=2,ncols=2,figsize=(10/3, 4))  # height can be anything
    diff = the_gal.get_band_pair('r-y').diff_image

    data = the_gal['r'].data - the_gal['y'].data
    to_diff_mask = the_gal.create_ellipse()
    mask_out_it = np.logical_not(to_diff_mask)
    
    ##bad_norm_diff = data * to_diff_mask##normalize_array(data,to_diff_mask)
    bad_norm_diff = normalize_matrix(data,to_diff_mask)

    diff[mask_out_it] = -np.Inf #to make background same color
    bad_norm_diff[mask_out_it] = -np.Inf #to make background same color
    #diff = bad_norm_diff

    shape = diff.shape
    cent = int(shape[0]/2),int(shape[1]/2)
    size = int(shape[0]/5),int(shape[1]/5)
    top_left = cent[0] - size[0]
    bottom_right = cent[0] + size[0]

    the_mask = the_gal[the_gal.ref_band].valid_pixel_mask
    pos_mask,neg_mask = the_gal.create_bisection()
    #m, s = np.mean(data[the_mask]), np.std(data[the_mask])
    
    cmap = create_color_map_class(pos_mask,neg_mask,np.logical_and(to_diff_mask,the_mask))

    import matplotlib.image as mpimg
    #pa_color = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\galaxies\\panstarrs\\{}\\{}\\{}_color.jfif".format(folder_name,the_gal.name,the_gal.name)
    #image = mpimg.imread(pa_color)
    image = mpimg.imread(get_color_image_path(name))
    
    ax[0,0].set_title('(a) color:')
    ax[0,0].imshow(image)
    ax[0,0].set_axis_off()
    
    ax[0,1].set_title('(b) reference band:')
    ref = the_gal['i'].data
    m, s = np.mean(ref[to_diff_mask]), np.std(ref[to_diff_mask])

    ax[0,1].imshow(ref[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[0,1].imshow(cmap[top_left:bottom_right,top_left:bottom_right], origin= 'lower',alpha=0.5)
    ax[0,1].set_axis_off()

    ax[1,0].set_title('(c) r-y:')
    data[mask_out_it] = -np.Inf #to make background same color
    im2 = ax[1,0].imshow(data[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[1,0].set_axis_off()

    ax[1,1].set_title('(d) difference:')
    im2 = ax[1,1].imshow(diff[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[1,1].set_axis_off()
    fig.tight_layout()

    
    pa = "E:\\grad_school\\research\\spin_parity_panstarrs\\paper_writing\\figure_maker_final\\{}_alt_norm_diff.png".format(the_gal.name)
    fig.savefig(pa, dpi=300)

def make_diff_image_example_for_paper(the_gal):
    #https://walmsley.dev/posts/typesetting-mnras-figures
    #https://support.microsoft.com/en-us/office/add-a-font-b7c5f17c-4426-4b53-967f-455339c564c1
    #error - font not appearing: https://stackoverflow.com/a/26106170
    ## delet file at: /mnt/c/Users/school/.matplotlib

    import matplotlib.pyplot as plt
    from matrix import normalize_matrix
    from visualize import create_color_map_class
    import numpy as np

    SMALL_SIZE = 9
    MEDIUM_SIZE = 9
    BIGGER_SIZE = 9

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)
    plt.rc('font', family='Nimbus Roman No9 L')

    fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(10/3, 2))  # height can be anything
    diff = the_gal.get_band_pair('r-y').diff_image
    to_diff_mask = the_gal.create_ellipse()
    mask_out_it = np.logical_not(to_diff_mask)
    diff[mask_out_it] = -np.Inf #to make background same color

    shape = diff.shape
    cent = int(shape[0]/2),int(shape[1]/2)
    size = int(shape[0]/11),int(shape[1]/11)
    top_left = cent[0] - size[0]
    bottom_right = cent[0] + size[0]

    import matplotlib.image as mpimg
    image = mpimg.imread(get_color_image_path(name))
    
    ax[0].set_title('(a) color:')
    ax[0].imshow(image)
    ax[0].set_axis_off()
    
    ax[1].set_title('(b) difference r-y:')
    ax[1].imshow(diff[top_left:bottom_right,top_left:bottom_right],origin='lower',cmap='plasma')
    ax[1].set_axis_off()
    fig.tight_layout()

    
    pa = "E:\\grad_school\\research\\spin_parity_panstarrs\\paper_writing\\figure_maker_final\\{}_diff.png".format(the_gal.name)
    fig.savefig(pa, dpi=300)

def make_example_histogram(the_gal):
    #https://walmsley.dev/posts/typesetting-mnras-figures
    #https://support.microsoft.com/en-us/office/add-a-font-b7c5f17c-4426-4b53-967f-455339c564c1
    #error - font not appearing: https://stackoverflow.com/a/26106170
    ## delet file at: /mnt/c/Users/school/.matplotlib

    import matplotlib.pyplot as plt
    from gofher import normalize_array
    from visualize import create_color_map_class
    import numpy as np

    SMALL_SIZE = 9
    MEDIUM_SIZE = 9
    BIGGER_SIZE = 9

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)
    plt.rc('font', family='Nimbus Roman No9 L')

    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10/3, 2))  # height can be anything
    band_pair = the_gal.get_band_pair('r-y')
    diff = the_gal.get_band_pair('r-y').diff_image
    to_diff_mask = the_gal.create_ellipse()
    mask_out_it = np.logical_not(to_diff_mask)
    diff[mask_out_it] = -np.Inf #to make background same color

    shape = diff.shape
    cent = int(shape[0]/2),int(shape[1]/2)
    size = int(shape[0]/11),int(shape[1]/11)
    top_left = cent[0] - size[0]
    bottom_right = cent[0] + size[0]

    import matplotlib.image as mpimg
    image = mpimg.imread(get_color_image_path(name))
    
    #ax[0].set_title('(a) color:')
    #ax[0].imshow(image)
    #ax[0].set_axis_off()

    both_sides = np.concatenate((band_pair.pos_side,band_pair.neg_side))
    mean = np.mean(both_sides)
    std = np.std(both_sides)
    hist_range = [mean-4*std,mean+4*std]
    
    #ax.set_title('(b) difference r-y:')
    pl = 'NW side'; nl = 'SE side' #temp
    ax.hist(band_pair.pos_side,bins=30,color='green',alpha=0.5, weights=np.ones_like(band_pair.pos_side) / len(band_pair.pos_side))
    ax.axvline(band_pair.pos_fit_norm_mean,color='green',label="{} mean = {:.3f}".format(pl,band_pair.pos_fit_norm_mean))

    ax.hist(band_pair.neg_side,bins=30,color='red',alpha=0.5, weights=np.ones_like(band_pair.neg_side) / len(band_pair.neg_side))
    ax.axvline(band_pair.neg_fit_norm_mean,color='red',label="{} mean = {:.3f}".format(nl,band_pair.neg_fit_norm_mean))
        
    #pos_x, pos_pdf, neg_x, neg_pdf = band_pair.evaluate_fit_norm()
    #ax.plot(pos_x,pos_pdf/pos_pdf.sum(),c='green',alpha=0.5 ,linestyle='dashed')
    #ax.plot(neg_x,neg_pdf/neg_pdf.sum(),c='red',alpha=0.5 ,linestyle='dashed')
    ax.set_xlim(hist_range)

    ax.legend(fontsize="6")

    ax.set_xlabel('Difference Value')
    ax.set_ylabel('Frequency')
    fig.tight_layout()

    
    pa = "E:\\grad_school\\research\\spin_parity_panstarrs\\paper_writing\\figure_maker_final\\{}_histogram.png".format(the_gal.name)
    fig.savefig(pa, dpi=300)

In [134]:
#make_diff_norm_image_example_for_paper(gal)
make_diff_image_example_for_paper(gal)